
<b>Mpipi model is a coarse-grain model for simulating IDP.</b> The original code is implemented in LAMMPS but I do not have access to the source code and am not familiar with LAMMPS either.



The original paper: <b> Physics-driven coarse-grained model for biomolecular phase separation with near-quantitative accuracy </b> Nature Computational Science volume 1, pages732–743 (2021)

PDF file can be accessed from [here](https://www.nature.com/articles/s43588-021-00155-3.epdf?sharing_token=EFzbweM1tOcYysI9oqUf9tRgN0jAjWel9jnR3ZoTv0OS6pLrbg60YiLYD1_wVdGp2Eu-H3ug5R37GhptvP-gObky-AytK6zbXFSGemcfdNg2kiUHIMYOv2d1wMOnNfHohUITqWql4RDHZjLEnHFs365Wh9a4RcPzXXly-vH42d8%3D)

This model is different from other hps-based model is instead of using LJ12-6 potential, it used Wang-Frenkel potential.

$\phi_{i,j}(r) = \epsilon_{i,j}  \alpha_{ij}  \left[\left(\frac{\sigma_{ij}}{r}\right)^{2\mu_{ij}}-1 \right]  \left[ \left(\frac{R_{ij}}{r}\right)^{2\mu_{ij}}-1\right]^{2\nu_{ij}}$

where, 

$\alpha_{ij}= 2\nu_{ij} \times \left(\frac{R_{ij}}{\sigma_{ij}}\right)^{2\mu_{ij}} \times \left[ \frac{(2\nu_{ij}+1)}{2\nu_{ij} \left( \left(\frac{R_{ij}}{\sigma_{ij}}\right)^{2\mu_{ij}}-1 \right) })\right]^{2\nu_{ij}+1}$

They used: $\nu_{ij}=1$ and $R_{ij}=3\sigma_{ij}$

$\sigma_{ij}, \epsilon_{ij}, \mu_{ij}$ are parameters specified for each pait of interacting beads.

# Implementation

<b>This post is for purpose of code-snippets which I implemented but it still not working. I will try to improve the post when I have time to make it clear.</b>

To implement this functional form. We used TabulatedFunction in OpenMM.

First, we need to define atom type of each residue, this definition is implemented in `hps/parameters/model_parameters.py` file as a dictionary:
```python
parameters = {
        
    "mpipi": {
        "bond_length": 0.381,
        "bonded_exclusions_index": 1,
        "MET": {
            "mass": 131.20,
            "charge": 0.0,
            "id": 0
        },
        "GLY": {
            "mass": 57.05,
            "charge": 0.0,
            "id": 1
        },
        "LYS": {
            "mass": 128.20,
            "charge": 0.75,
            "id": 2
        },
        "THR": {
            "mass": 101.10,
            "charge": 0.0,
            "id": 3
        },
        "ARG": {
            "mass": 156.20,
            "charge": 0.75,
            "id": 4
        },
        "ALA": {
            "mass": 71.08,
            "charge": 0.0,
            "id": 5
        },
        "ASP": {
            "mass": 115.10,
            "charge": -0.75,
            "id": 6
        },
        "GLU": {
            "mass": 129.10,
            "charge": -0.75,
            "id": 7
        },
        "TYR": {
            "mass": 163.20,
            "charge": 0.0,
            "id": 8
        },
        "VAL": {
            "mass": 99.07,
            "charge": 0.0,
            "id": 9
        },
        "LEU": {
            "mass": 113.20,
            "charge": 0.0,
            "id": 10
        },
        "GLN": {
            "mass": 128.10,
            "charge": 0.0,
            "id": 11
        },
        "TRP": {
            "mass": 186.20,
            "charge": 0.0,
            "id": 12
        },
        "PHE": {
            "mass": 147.20,
            "charge": 0.0,
            "id": 13
        },
        "SER": {
            "mass": 87.08,
            "charge": 0.0,
            "id": 14
        },
        "HIS": {
            "mass": 137.10,
            "charge": 0.375,
            "id": 15
        },
        "ASN": {
            "mass": 114.10,
            "charge": 0.0,
            "id": 16
        },
        "PRO": {
            "mass": 97.12,
            "charge": 0.0,
            "id": 17
        },
        "CYS": {
            "mass": 103.10,
            "charge": 0.0,
            "id": 18
        },
        "ILE": {
            "mass": 113.20,
            "charge": 0.0,
            "id": 19
        }

    }
}

```


--------------------------------------------

<b> $\epsilon_{ij}$ pair parameters</b>:

```python
table_eps = np.array([[0.165536, 0.284583, 0.136758, 0.147114, 0.717619,
                               0.186280, 0.306152, 0.324582, 0.959705, 0.094437,
                               0.105776, 0.502105, 1.233991, 0.902083, 0.211635,
                               0.851406, 0.488298, 0.247358, 0.227764, 0.083592],
                              [0.284583, 0.403630, 0.255806, 0.266161, 0.836666,
                               0.305327, 0.425199, 0.443630, 1.078752, 0.213484,
                               0.224823, 0.621152, 1.353039, 1.021130, 0.330682,
                               0.970454, 0.607345, 0.366405, 0.346812, 0.202639],
                              [0.136758, 0.255806, 0.079986, 0.118336, 0.510189,
                               0.157502, 0.002063, 0.002201, 0.429701, 0.065660,
                               0.076998, 0.473328, 0.438466, 0.482315, 0.182858,
                               0.365556, 0.459520, 0.218581, 0.198987, 0.054815],
                              [0.147114, 0.266161, 0.118336, 0.128691, 0.699197,
                               0.167858, 0.287729, 0.306160, 0.941283, 0.076015,
                               0.087354, 0.483683, 1.215565, 0.883661, 0.193213,
                               0.832984, 0.469876, 0.228936, 0.209342, 0.065170],
                              [0.717619, 0.836666, 0.510189, 0.699197, 0.376209,
                               0.738363, 0.006368, 0.006502, 2.543119, 0.646520,
                               0.657859, 1.054188, 2.893893, 2.280037, 0.763718,
                               0.519808, 1.040381, 0.799441, 0.779847, 0.635675],
                              [0.186280, 0.305327, 0.157502, 0.167858, 0.738363,
                               0.207024, 0.326896, 0.345326, 0.980449, 0.115181,
                               0.126520, 0.522849, 1.254736, 0.922827, 0.232379,
                               0.872151, 0.509046, 0.268102, 0.248509, 0.104336],
                              [0.306152, 0.425199, 0.002063, 0.287729, 0.006368,
                               0.326896, 0.330938, 0.344590, 1.100321, 0.235049,
                               0.246387, 0.642721, 1.374603, 1.042699, 0.352247,
                               0.007355, 0.628914, 0.387974, 0.368380, 0.224208],
                              [0.324582, 0.443630, 0.002201, 0.306160, 0.006502,
                               0.345326, 0.344590, 0.358242, 1.118751, 0.253479,
                               0.264818, 0.661151, 1.393034, 1.061129, 0.370677,
                               0.007494, 0.647344, 0.406404, 0.386811, 0.242639],
                              [0.959705, 1.078752, 0.429701, 0.941283, 2.543119,
                               0.980449, 1.100321, 1.118751, 1.753874, 0.888606,
                               0.899945, 1.296274, 2.028156, 1.696252, 1.005804,
                               1.645576, 1.282467, 1.041527, 1.021934, 0.877761],
                              [0.094437, 0.213484, 0.065660, 0.076015, 0.646520,
                               0.115181, 0.235049, 0.253479, 0.888606, 0.023338,
                               0.034677, 0.431006, 1.162888, 0.830984, 0.140536,
                               0.780308, 0.417199, 0.176259, 0.156666, 0.012493],
                              [0.105776, 0.224823, 0.076998, 0.087354, 0.657859,
                               0.126520, 0.246387, 0.264818, 0.899945, 0.034677,
                               0.046016, 0.442345, 1.174227, 0.842323, 0.151875,
                               0.791646, 0.428538, 0.187598, 0.168004, 0.023832],
                              [0.502105, 0.621152, 0.473328, 0.483683, 1.054188,
                               0.522849, 0.642721, 0.661151, 1.296274, 0.431006,
                               0.442345, 0.838674, 1.570556, 1.238652, 0.548204,
                               1.187976, 0.824867, 0.583927, 0.564334, 0.420161],
                              [1.233991, 1.353039, 0.438466, 1.215565, 2.893893,
                               1.254736, 1.374603, 1.393034, 2.028156, 1.162888,
                               1.174227, 1.570556, 2.302443, 1.970538, 1.280086,
                               1.919858, 1.556753, 1.315814, 1.296220, 1.152048],
                              [0.902083, 1.021130, 0.482315, 0.883661, 2.280037,
                               0.922827, 1.042699, 1.061129, 1.696252, 0.830984,
                               0.842323, 1.238652, 1.970538, 1.638630, 0.948182,
                               1.587954, 1.224845, 0.983905, 0.964312, 0.820139],
                              [0.211635, 0.330682, 0.182858, 0.193213, 0.763718,
                               0.232379, 0.352247, 0.370677, 1.005804, 0.140536,
                               0.151875, 0.548204, 1.280086, 0.948182, 0.257734,
                               0.897506, 0.534397, 0.293457, 0.273864, 0.129691],
                              [0.851406, 0.970454, 0.365556, 0.832984, 0.519808,
                               0.872151, 0.007355, 0.007494, 1.645576, 0.780308,
                               0.791646, 1.187976, 1.919858, 1.587954, 0.897506,
                               0.113872, 1.174168, 0.933229, 0.913635, 0.769463],
                              [0.488298, 0.607345, 0.459520, 0.469876, 1.040381,
                               0.509046, 0.628914, 0.647344, 1.282467, 0.417199,
                               0.428538, 0.824867, 1.556753, 1.224845, 0.534397,
                               1.174168, 0.811064, 0.570124, 0.550531, 0.406358],
                              [0.247358, 0.366405, 0.218581, 0.228936, 0.799441,
                               0.268102, 0.387974, 0.406404, 1.041527, 0.176259,
                               0.187598, 0.583927, 1.315814, 0.983905, 0.293457,
                               0.933229, 0.570124, 0.329185, 0.309591, 0.165419],
                              [0.227764, 0.346812, 0.198987, 0.209342, 0.779847,
                               0.248509, 0.368380, 0.386811, 1.021934, 0.156666,
                               0.168004, 0.564334, 1.296220, 0.964312, 0.273864,
                               0.913635, 0.550531, 0.309591, 0.289997, 0.145825],
                              [0.083592, 0.202639, 0.054815, 0.065170, 0.635675,
                               0.104336, 0.224208, 0.242639, 0.877761, 0.012493,
                               0.023832, 0.420161, 1.152048, 0.820139, 0.129691,
                               0.769463, 0.406358, 0.165419, 0.145825, 0.001653]])
        table_eps_ravel = table_eps.ravel().tolist()

```

This table has a shape of (20,20)- 20 rows and 20 columns. The order of rows and columns follows the `id` in the residue definition above.


Each element of the matrix is the $\epsilon_{ij}$ parameter between residue type `i` and type `j`. For example, `table_eps[0,0]` is the interaction between residue type `0` and `0`, which corresponds to the `MET-MET` interaction. Similarly, `table_eps[19,0]` is interaction between ILE-MET

------------------------------------

Using the same method, we define $\sigma$ matrix:
```python
table_sigma = np.array([[0.646795, 0.557618, 0.656778, 0.617823, 0.664396,
                                 0.586850, 0.614146, 0.631818, 0.659004, 0.632057,
                                 0.648344, 0.636519, 0.675573, 0.653821, 0.593894,
                                 0.639251, 0.618802, 0.613446, 0.609429, 0.649639],
                                [0.557618, 0.469511, 0.567134, 0.528442, 0.576639,
                                 0.498013, 0.525925, 0.543637, 0.571317, 0.541239,
                                 0.557893, 0.548630, 0.587924, 0.566122, 0.505252,
                                 0.551541, 0.530902, 0.524999, 0.520872, 0.558033],
                                [0.656778, 0.567134, 0.667134, 0.627940, 0.673819,
                                 0.596669, 0.623699, 0.641358, 0.668408, 0.643354,
                                 0.659256, 0.645969, 0.684969, 0.663228, 0.603618,
                                 0.648660, 0.628255, 0.623106, 0.619115, 0.661800],
                                [0.617823, 0.528442, 0.627940, 0.588906, 0.635202,
                                 0.557795, 0.584987, 0.602654, 0.629806, 0.603697,
                                 0.619809, 0.607329, 0.646378, 0.624625, 0.564801,
                                 0.610053, 0.589610, 0.584320, 0.580321, 0.621462],
                                [0.664396, 0.576639, 0.673819, 0.635202, 0.683905,
                                 0.604909, 0.632986, 0.650709, 0.678624, 0.647780,
                                 0.664461, 0.655828, 0.695255, 0.673424, 0.612191,
                                 0.658837, 0.638103, 0.631985, 0.627828, 0.664491],
                                [0.586850, 0.498013, 0.596669, 0.557795, 0.604909,
                                 0.527007, 0.554510, 0.572198, 0.599537, 0.571467,
                                 0.587918, 0.576988, 0.616122, 0.594352, 0.534119,
                                 0.579777, 0.559268, 0.553738, 0.549687, 0.588726],
                                [0.614146, 0.525925, 0.623699, 0.584987, 0.632986,
                                 0.554510, 0.582352, 0.600058, 0.627647, 0.597874,
                                 0.614509, 0.605004, 0.644244, 0.622456, 0.561730,
                                 0.607876, 0.587280, 0.581455, 0.577340, 0.614699],
                                [0.631818, 0.543637, 0.641358, 0.602654, 0.650709,
                                 0.572198, 0.600058, 0.617767, 0.645370, 0.615504,
                                 0.632146, 0.622721, 0.661969, 0.640180, 0.579422,
                                 0.625600, 0.604999, 0.599152, 0.595036, 0.632333],
                                [0.659004, 0.571317, 0.668408, 0.629806, 0.678624,
                                 0.599537, 0.627647, 0.645370, 0.673363, 0.642355,
                                 0.659039, 0.650525, 0.690005, 0.668159, 0.606828,
                                 0.653569, 0.632798, 0.626630, 0.622465, 0.659055],
                                [0.632057, 0.541239, 0.643354, 0.603697, 0.647780,
                                 0.571467, 0.597874, 0.615504, 0.642355, 0.626600,
                                 0.639093, 0.619973, 0.658912, 0.637178, 0.578166,
                                 0.622610, 0.602261, 0.597433, 0.593567, 0.671801],
                                [0.648344, 0.557893, 0.659256, 0.619809, 0.664461,
                                 0.587918, 0.614509, 0.632146, 0.659039, 0.639093,
                                 0.653407, 0.636649, 0.675594, 0.653860, 0.594697,
                                 0.639294, 0.618936, 0.614021, 0.610125, 0.660772],
                                [0.636519, 0.548630, 0.645969, 0.607329, 0.655828,
                                 0.576988, 0.605004, 0.622721, 0.650525, 0.619973,
                                 0.636649, 0.627785, 0.667141, 0.645328, 0.584255,
                                 0.630743, 0.610059, 0.604035, 0.599885, 0.636715],
                                [0.675573, 0.587924, 0.684969, 0.646378, 0.695255,
                                 0.616122, 0.644244, 0.661969, 0.690005, 0.658912,
                                 0.675594, 0.667141, 0.706655, 0.684798, 0.623414,
                                 0.670209, 0.649416, 0.643217, 0.639053, 0.675603],
                                [0.653821, 0.566122, 0.663228, 0.624625, 0.673424,
                                 0.594352, 0.622456, 0.640180, 0.668159, 0.637178,
                                 0.653860, 0.645328, 0.684798, 0.662955, 0.601640,
                                 0.648367, 0.627603, 0.621441, 0.617279, 0.653879],
                                [0.593894, 0.505252, 0.603618, 0.564801, 0.612191,
                                 0.534119, 0.561730, 0.579422, 0.606828, 0.578166,
                                 0.594697, 0.584255, 0.623414, 0.601640, 0.541267,
                                 0.587063, 0.566533, 0.560932, 0.556848, 0.595253],
                                [0.639251, 0.551541, 0.648660, 0.610053, 0.658837,
                                 0.579777, 0.607876, 0.625600, 0.653569, 0.622610,
                                 0.639294, 0.630743, 0.670209, 0.648367, 0.587063,
                                 0.633778, 0.613018, 0.606865, 0.602702, 0.639315],
                                [0.618802, 0.530902, 0.628255, 0.589610, 0.638103,
                                 0.559268, 0.587280, 0.604999, 0.632798, 0.602261,
                                 0.618936, 0.610059, 0.649416, 0.627603, 0.566533,
                                 0.613018, 0.592335, 0.586308, 0.582164, 0.618999],
                                [0.613446, 0.524999, 0.623106, 0.584320, 0.631985,
                                 0.553738, 0.581455, 0.599152, 0.626630, 0.597433,
                                 0.614021, 0.604035, 0.643217, 0.621441, 0.560932,
                                 0.606865, 0.586308, 0.580616, 0.576515, 0.614386],
                                [0.609429, 0.520872, 0.619115, 0.580321, 0.627828,
                                 0.549687, 0.577340, 0.595036, 0.622465, 0.593567,
                                 0.610125, 0.599885, 0.639053, 0.617279, 0.556848,
                                 0.602702, 0.582164, 0.576515, 0.572436, 0.610587],
                                [0.649639, 0.558033, 0.661800, 0.621462, 0.664491,
                                 0.588726, 0.614699, 0.632333, 0.659055, 0.671801,
                                 0.660772, 0.636715, 0.675603, 0.653879, 0.595253,
                                 0.639315, 0.618999, 0.614386, 0.610587, 0.692168]])
        table_sigma_ravel = table_sigma.ravel().tolist()
```

<b> $\nu_{ij}$ matrix:</b>

As said above, $\nu_{ij}$ is one for all. But for generalize in case we want to modify, I till defined this matrix in the same fashion.

```python
table_nu = np.array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.],
                             [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                              1., 1., 1., 1.]])
        table_nu_ravel = table_nu.ravel().tolist()
```

---------------------------------------------

<b> $\mu_{ij}$ matrix:</b>
```python
table_mu = np.array([[2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 4.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 2.],
                            [2., 2., 2., 2., 2., 2., 2., 2., 2., 4., 2., 2., 2.,
                             2., 2., 2., 2., 2., 2., 11.]])
        table_mu_ravel = table_mu.ravel().tolist()
```
-------------------------------------
<b> $R_{ij}$ matrix:</b>

This is simplely $3\sigma_{ij}$
```python
table_rc = np.array([[1.940385, 1.672854, 1.970335, 1.853468, 1.993189,
                              1.760551, 1.842438, 1.895453, 1.977011, 1.896171,
                              1.945033, 1.909556, 2.026719, 1.961463, 1.781682,
                              1.917752, 1.856406, 1.840339, 1.828288, 1.948918],
                             [1.672854, 1.408533, 1.701401, 1.585327, 1.729916,
                              1.494040, 1.577776, 1.630912, 1.713950, 1.623718,
                              1.673679, 1.645890, 1.763771, 1.698366, 1.515757,
                              1.654623, 1.592705, 1.574997, 1.562617, 1.674099],
                             [1.970335, 1.701401, 2.001402, 1.883820, 2.021458,
                              1.790006, 1.871096, 1.924073, 2.005225, 1.930062,
                              1.977767, 1.937908, 2.054906, 1.989684, 1.810854,
                              1.945981, 1.884764, 1.869318, 1.857346, 1.985399],
                             [1.853468, 1.585327, 1.883820, 1.766718, 1.905605,
                              1.673384, 1.754960, 1.807962, 1.889419, 1.811090,
                              1.859427, 1.821986, 1.939133, 1.873875, 1.694402,
                              1.830159, 1.768830, 1.752959, 1.740963, 1.864385],
                             [1.993189, 1.729916, 2.021458, 1.905605, 2.051715,
                              1.814728, 1.898958, 1.952126, 2.035872, 1.943340,
                              1.993382, 1.967484, 2.085765, 2.020273, 1.836572,
                              1.976511, 1.914308, 1.895955, 1.883483, 1.993474],
                             [1.760551, 1.494040, 1.790006, 1.673384, 1.814728,
                              1.581022, 1.663531, 1.716595, 1.798612, 1.714402,
                              1.763755, 1.730964, 1.848366, 1.783057, 1.602356,
                              1.739330, 1.677803, 1.661215, 1.649060, 1.766179],
                             [1.842438, 1.577776, 1.871096, 1.754960, 1.898958,
                              1.663531, 1.747057, 1.800175, 1.882942, 1.793621,
                              1.843527, 1.815012, 1.932732, 1.867367, 1.685190,
                              1.823628, 1.761839, 1.744364, 1.732021, 1.844096],
                             [1.895453, 1.630912, 1.924073, 1.807962, 1.952126,
                              1.716595, 1.800175, 1.853301, 1.936109, 1.846512,
                              1.896437, 1.868164, 1.985908, 1.920540, 1.738266,
                              1.876800, 1.814997, 1.797457, 1.785109, 1.896998],
                             [1.977011, 1.713950, 2.005225, 1.889419, 2.035872,
                              1.798612, 1.882942, 1.936109, 2.020090, 1.927065,
                              1.977116, 1.951575, 2.070015, 2.004477, 1.820484,
                              1.960707, 1.898395, 1.879889, 1.867395, 1.977165],
                             [1.896171, 1.623718, 1.930062, 1.811090, 1.943340,
                              1.714402, 1.793621, 1.846512, 1.927065, 1.879800,
                              1.917280, 1.859919, 1.976735, 1.911535, 1.734499,
                              1.867831, 1.806782, 1.792299, 1.780701, 2.015403],
                             [1.945033, 1.673679, 1.977767, 1.859427, 1.993382,
                              1.763755, 1.843527, 1.896437, 1.977116, 1.917280,
                              1.960222, 1.909947, 2.026781, 1.961581, 1.784091,
                              1.917883, 1.856809, 1.842064, 1.830374, 1.982317],
                             [1.909556, 1.645890, 1.937908, 1.821986, 1.967484,
                              1.730964, 1.815012, 1.868164, 1.951575, 1.859919,
                              1.909947, 1.883355, 2.001422, 1.935983, 1.752766,
                              1.892230, 1.830178, 1.812104, 1.799656, 1.910146],
                             [2.026719, 1.763771, 2.054906, 1.939133, 2.085765,
                              1.848366, 1.932732, 1.985908, 2.070015, 1.976735,
                              2.026781, 2.001422, 2.119964, 2.054395, 1.870242,
                              2.010627, 1.948249, 1.929652, 1.917160, 2.026810],
                             [1.961463, 1.698366, 1.989684, 1.873875, 2.020273,
                              1.783057, 1.867367, 1.920540, 2.004477, 1.911535,
                              1.961581, 1.935983, 2.054395, 1.988865, 1.804920,
                              1.945102, 1.882808, 1.864322, 1.851836, 1.961636],
                             [1.781682, 1.515757, 1.810854, 1.694402, 1.836572,
                              1.602356, 1.685190, 1.738266, 1.820484, 1.734499,
                              1.784091, 1.752766, 1.870242, 1.804920, 1.623801,
                              1.761190, 1.699598, 1.682795, 1.670544, 1.785760],
                             [1.917752, 1.654623, 1.945981, 1.830159, 1.976511,
                              1.739330, 1.823628, 1.876800, 1.960707, 1.867831,
                              1.917883, 1.892230, 2.010627, 1.945102, 1.761190,
                              1.901335, 1.839054, 1.820595, 1.808106, 1.917944],
                             [1.856406, 1.592705, 1.884764, 1.768830, 1.914308,
                              1.677803, 1.761839, 1.814997, 1.898395, 1.806782,
                              1.856809, 1.830178, 1.948249, 1.882808, 1.699598,
                              1.839054, 1.777005, 1.758925, 1.746493, 1.856996],
                             [1.840339, 1.574997, 1.869318, 1.752959, 1.895955,
                              1.661215, 1.744364, 1.797457, 1.879889, 1.792299,
                              1.842064, 1.812104, 1.929652, 1.864322, 1.682795,
                              1.820595, 1.758925, 1.741847, 1.729545, 1.843158],
                             [1.828288, 1.562617, 1.857346, 1.740963, 1.883483,
                              1.649060, 1.732021, 1.785109, 1.867395, 1.780701,
                              1.830374, 1.799656, 1.917160, 1.851836, 1.670544,
                              1.808106, 1.746493, 1.729545, 1.717309, 1.831760],
                             [1.948918, 1.674099, 1.985399, 1.864385, 1.993474,
                              1.766179, 1.844096, 1.896998, 1.977165, 2.015403,
                              1.982317, 1.910146, 2.026810, 1.961636, 1.785760,
                              1.917944, 1.856996, 1.843158, 1.831760, 2.076504]])
        table_rc_ravel = table_rc.ravel().tolist()
```

# TabulatedFunction:

This is an interesting point from computational perpective.

More information about TabulatedFunction can be found in OpenMM documentation [here](http://docs.openmm.org/7.2.0/api-c++/generated/OpenMM.Discrete2DFunction.html)

Here, I will use 2D Discrete function. 

The function works as follow:
`Discrete2DFunction(xsize, ysize, val)`

the tabulated values of the function `f(x,y)`, ordered so that `values[i+xsize*j] = f(i,j)`. This must be of length `xsize*ysize`.

This is the reason why I use `ravel()` function from python to covnert 2D array to 1D array above.

# The main part of Wang-Frenkel Potential:

```python
def add_Wang_Frenkel_Forces(self, use_pbc: bool) ):
    # number of atom types in model. currently with protein, there are 20.
    n_atom_types = table_sigma.shape[0]

    # eps, sigma, nu, mu, rc: load from tabular table
    energy_function = 'eps * 2*nu*(rc/sigma)^(2*mu) * ((2*nu+1)/(2*nu*((rc/sigma)^(2*mu)-1)) )^(2*nu+1)'
    energy_function += '* ((sigma/r)^(2*mu)-1 ) * ((rc/r)^(2*mu)-1)^(2*nu);'
    energy_function += 'eps = eps_table(id1, id2); sigma = sigma_table(id1, id2);'
    energy_function += 'nu = nu_table(id1, id2); mu = mu_table(id1, id2);'
    energy_function += 'rc=rc_table(id1, id2)'

    self.wang_Frenkel_Force = openmm.CustomNonbondedForce(energy_function)
    self.wang_Frenkel_Force.addTabulatedFunction('eps_table', openmm.Discrete2DFunction(n_atom_types, n_atom_types,
                                                                                        table_eps_ravel))
    self.wang_Frenkel_Force.addTabulatedFunction('sigma_table',
                                                 openmm.Discrete2DFunction(n_atom_types, n_atom_types,
                                                                           table_sigma_ravel))
    self.wang_Frenkel_Force.addTabulatedFunction('nu_table', openmm.Discrete2DFunction(n_atom_types, n_atom_types,
                                                                                       table_nu_ravel))
    self.wang_Frenkel_Force.addTabulatedFunction('mu_table', openmm.Discrete2DFunction(n_atom_types, n_atom_types,
                                                                                       table_mu_ravel))
    self.wang_Frenkel_Force.addTabulatedFunction('rc_table', openmm.Discrete2DFunction(n_atom_types, n_atom_types,
                                                                                       table_rc_ravel))
    self.wang_Frenkel_Force.addPerParticleParameter('id')

    for i, atom in enumerate(self.atoms):
        self.wang_Frenkel_Force.addParticle((self.particle_type_id[i],))

    if use_pbc:
        self.wang_Frenkel_Force.setNonbondedMethod(openmm.NonbondedForce.CutoffPeriodic)
    else:
        self.wang_Frenkel_Force.setNonbondedMethod(openmm.NonbondedForce.CutoffNonPeriodic)

    self.wang_Frenkel_Force.setCutoffDistance(wang_frenkel_cutoff)



    # set exclusion rule
    bonded_exclusions = [(b[0].index, b[1].index) for b in list(self.topology.bonds())]
    self.wang_Frenkel_Force.createExclusionsFromBonds(bonded_exclusions, self.bonded_exclusions_index)
```

In this code, I call:\
$\epsilon_{ij}$ = `eps`,\
$\nu_{ij}$ = `nu`,\
$\mu_{ij}$ = `mu`,\
$R_{ij}$ = `rc`

Each particle has Per-Particle-Parameter: `id`

Here, we define, i.e: `eps` is a tabulated function: `eps = eps_table(id1, id2)`. The value will be extracted from particle `id`

<b> Currently, the implementation return `NaN` pair-wise energy. The problem may be due to the initial conformation of protein since I test with a system of 7 particles it can calculate correctly, but for larger systems, it's hard to debug so just STOP there.</b>